# Calculating Forest Loss (using IDEAM)

In [9]:
import os
import ee
import geemap 
import pandas as pd

#ee.Authenticate()
ee.Initialize()

In [10]:
# Define the bounding box of Colombia (using a shapefile uploaded to the assets folder)
colombiaMpios = ee.FeatureCollection('projects/ee-juamiji/assets/Muni')

# Import the Forest loss image
flossHansen = ee.Image("UMD/hansen/global_forest_change_2023_v1_11")

# Import the Primary Tropical Forest cover image collection and filter it for Colombia
primaryForest_ideam = ee.Image("projects/ee-juamiji/assets/SouthAmerica_2001_primary").selfMask()
primaryForest_ideam_COL = primaryForest_ideam.clip(colombiaMpios)

# Clip the Forest loss image to Colombia's boundaries
flossHansen_COL = flossHansen.clip(colombiaMpios)


In [11]:
# Visualization parameters for the map
viz_params = {
    'min':1,  # Minimum value for visualization
    'max': 1,  # Maximum value for visualization
    'palette': ['008000']  # Color gradient
}

# Initialize the map using geemap
Map = geemap.Map(center=[4.0, -72.0], zoom=5)  # Center roughly in Colombia with an appropriate zoom level
Map.addLayer(primaryForest_ideam_COL, viz_params, "Primary Tropical Forests")
Map

Map(center=[4.0, -72.0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(…

In [12]:
# Create a mask where fcover is equal to 1.
primaryMask = primaryForest_ideam_COL.eq(1)

# Masking the floss imagery using the primary mask (cover equals 1)
flossHansen_COLmasked = flossHansen_COL.updateMask(primaryMask)

# Select the 'lossyear' band from the masked forest loss image
loss_year = flossHansen_COLmasked.select(['lossyear'])

## Code for all years from 2000 to 2023:

In [13]:
# Calculate the total loss area per municipality
def calculate_loss_area(feature, year):

    loss_year_mask = loss_year.eq(year)

    # Calculate the loss area for the current year in square meters
    loss_area_image = loss_year_mask.multiply(ee.Image.pixelArea())

    # Use reduceRegion to calculate the sum of the loss area within the feature's geometry
    loss_area = loss_area_image.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=feature.geometry(),
        scale=30,
        maxPixels=1e9
    ).get('lossyear')  # Replace 'lossyear' with the correct property name if different
    
     # Dynamically name the loss area property
    loss_area_property_name = f'lossarea{year}'

    # Return a new feature with only the 'lossArea' and 'codmpio' properties
    return ee.Feature(None, {
        loss_area_property_name: loss_area,
        'codmpio': feature.get('codmpio'),  # Ensure 'codmpio' exists in the original feature
    })

# Dictionary to store results for each year
year_results_dict = {}

for year in range(0, 24):
    # Use a lambda function to pass the current year into the map function
    current_year_results = colombiaMpios.map(lambda feature: calculate_loss_area(feature, year))
    year_results_dict[f'year_results{year}'] = current_year_results
    # Optionally print the first feature's info to check
    print(year_results_dict[f'year_results{year}'].first().getInfo())

{'type': 'Feature', 'geometry': None, 'id': '00000000000000000263', 'properties': {'codmpio': 5002, 'lossarea0': 0}}
{'type': 'Feature', 'geometry': None, 'id': '00000000000000000263', 'properties': {'codmpio': 5002, 'lossarea1': 0}}
{'type': 'Feature', 'geometry': None, 'id': '00000000000000000263', 'properties': {'codmpio': 5002, 'lossarea2': 0}}
{'type': 'Feature', 'geometry': None, 'id': '00000000000000000263', 'properties': {'codmpio': 5002, 'lossarea3': 0}}
{'type': 'Feature', 'geometry': None, 'id': '00000000000000000263', 'properties': {'codmpio': 5002, 'lossarea4': 0}}
{'type': 'Feature', 'geometry': None, 'id': '00000000000000000263', 'properties': {'codmpio': 5002, 'lossarea5': 0}}
{'type': 'Feature', 'geometry': None, 'id': '00000000000000000263', 'properties': {'codmpio': 5002, 'lossarea6': 1778.8826904296875}}
{'type': 'Feature', 'geometry': None, 'id': '00000000000000000263', 'properties': {'codmpio': 5002, 'lossarea7': 0}}
{'type': 'Feature', 'geometry': None, 'id': '00

In [14]:
# Define a function to export FeatureCollections to Google Drive
def export_to_drive(feature_collection, year, description_prefix="ForestLossIdeamData", folder_name="EarthEngineExports"):
    # Create the export task
    export_task = ee.batch.Export.table.toDrive(
        collection=feature_collection,
        description=f"{description_prefix}_Year{year}",
        folder="EarthEngineExports",
        fileNamePrefix=f"ForestLoss_IDEAM_Year{year}",
        fileFormat="CSV"
    )
    # Start the export task
    export_task.start()
    print(f"Exporting {description_prefix}_Year{year} to Google Drive...")

# Loop through each item in the dictionary and initiate an export
for year, feature_collection in year_results_dict.items():
    # Extract the year from the key (which is a string like 'year_results11')
    extracted_year = int(year.replace('year_results', ''))
    export_to_drive(feature_collection, extracted_year)

Exporting ForestLossIdeamData_Year0 to Google Drive...
Exporting ForestLossIdeamData_Year1 to Google Drive...
Exporting ForestLossIdeamData_Year2 to Google Drive...
Exporting ForestLossIdeamData_Year3 to Google Drive...
Exporting ForestLossIdeamData_Year4 to Google Drive...
Exporting ForestLossIdeamData_Year5 to Google Drive...
Exporting ForestLossIdeamData_Year6 to Google Drive...
Exporting ForestLossIdeamData_Year7 to Google Drive...
Exporting ForestLossIdeamData_Year8 to Google Drive...
Exporting ForestLossIdeamData_Year9 to Google Drive...
Exporting ForestLossIdeamData_Year10 to Google Drive...
Exporting ForestLossIdeamData_Year11 to Google Drive...
Exporting ForestLossIdeamData_Year12 to Google Drive...
Exporting ForestLossIdeamData_Year13 to Google Drive...
Exporting ForestLossIdeamData_Year14 to Google Drive...
Exporting ForestLossIdeamData_Year15 to Google Drive...
Exporting ForestLossIdeamData_Year16 to Google Drive...
Exporting ForestLossIdeamData_Year17 to Google Drive...
Ex

In [15]:
# Convert the FeatureCollection to a list of features
features =  year_results_dict['year_results3'].getInfo()['features']

# Extract properties from each feature
data = [feature['properties'] for feature in features]

# Create a Pandas DataFrame
df = pd.DataFrame(data)

# Display the first few rows of the DataFrame
print(df.head())

   codmpio      lossarea3
0     5002       0.000000
1     5004   63062.670898
2    50006   45476.478882
3    27006  444951.373779
4    41006    6255.092163


In [16]:
# Calculate basic statistics
summary_stats = {
    'N': len(df),  # Number of observations
    'mean': df['lossarea3'].mean(),  # Mean
    'sd': df['lossarea3'].std(),  # Standard deviation
    'min': df['lossarea3'].min(),  # Minimum
    'p50': df['lossarea3'].median(),  # Median (50th percentile)
    'max': df['lossarea3'].max()  # Maximum
}

# Convert to a DataFrame for better visualization
summ_df = pd.DataFrame(summary_stats, index=[0])

# Display the summary statistics
print(summ_df)

      N           mean            sd  min  p50           max
0  1120  286596.013138  1.710529e+06  0.0  0.0  2.802999e+07
